# DQN Implementation for LunarLander-v2

This notebook demonstrates how to implement a Deep Q-Network (DQN) to solve the **LunarLander-v2** environment in Gymnasium.  
We organize the code so that:
- **`dqn_model.py`** contains the Q-Network definition.
- **`dqn_agent.py`** contains the replay buffer and DQN agent logic.
- This notebook loads those modules and runs training, then plots reward over time.


In [ ]:
!pip install --quiet gymnasium[box2d] torch matplotlib


In [ ]:
import gymnasium as gym
import numpy as np
import torch
import matplotlib.pyplot as plt

# Ensure our modules are in the Python path
import sys
sys.path.append('/mnt/data')

from dqn_agent import DQNAgent


In [ ]:
# Hyperparameters
ENV_NAME     = "LunarLander-v2"
BUFFER_SIZE  = int(1e5)
BATCH_SIZE   = 64
GAMMA        = 0.99
LR           = 1e-3
TAU          = 1e-3
UPDATE_EVERY = 4
EPS_START    = 1.0
EPS_END      = 0.01
EPS_DECAY    = 0.995
HIDDEN_DIM   = 128
MAX_EPISODES = 500
PRINT_EVERY  = 20


In [ ]:
def train_dqn(env_name, agent, max_episodes=MAX_EPISODES, print_every=PRINT_EVERY):
    env = gym.make(env_name)
    scores = []
    scores_window = deque(maxlen=print_every)
    eps = EPS_START

    for i_episode in range(1, max_episodes + 1):
        state, _ = env.reset()
        score = 0
        done = False
        while not done:
            action = agent.act(state, eps)
            next_state, reward, terminated, truncated, _ = env.step(action)
            done = terminated or truncated
            agent.step(state, action, reward, next_state, done)
            state = next_state
            score += reward

        scores_window.append(score)
        scores.append(score)
        eps = max(EPS_END, EPS_DECAY * eps)

        if i_episode % print_every == 0:
            print(f"Episode {i_episode}	Average Score: {np.mean(scores_window):.2f}")

    env.close()
    return scores

# Instantiate agent and train
env_tmp = gym.make(ENV_NAME)
state_dim = env_tmp.observation_space.shape[0]
action_dim = env_tmp.action_space.n
env_tmp.close()

agent = DQNAgent(
    state_dim=state_dim,
    action_dim=action_dim,
    seed=0,
    buffer_size=BUFFER_SIZE,
    batch_size=BATCH_SIZE,
    gamma=GAMMA,
    lr=LR,
    tau=TAU,
    update_every=UPDATE_EVERY,
    device='cpu'
)

scores = train_dqn(ENV_NAME, agent)


In [ ]:
# Plot the scores over episodes
plt.figure(figsize=(10,5))
plt.plot(scores, label="Score per Episode")
# Running average (window size = PRINT_EVERY)
running_avg = np.convolve(scores, np.ones(PRINT_EVERY)/PRINT_EVERY, mode='valid')
plt.plot(np.arange(len(running_avg)) + PRINT_EVERY - 1, running_avg, color='orange', label=f"{PRINT_EVERY}-episode avg")
plt.xlabel("Episode")
plt.ylabel("Score")
plt.title("DQN on LunarLander-v2")
plt.legend()
plt.show()
